# CAICT 風機結冰預測 - Data Exploration

Use FACETS

2017/07/20
http://www.industrial-bigdata.com/competition/competitionAction!showDetail.action?competition.competitionId=1

# **1. Load Data and Modules**

**Load Python modules:**

In [1]:
%matplotlib inline
import sys
sys.path.append('/Users/jayhsu/work/github/facets/facets_overview/python')
import numpy as np
import pandas as pd
import datetime
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
import shutil
import csv
import sklearn as sk
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import *
from IPython.core.interactiveshell import InteractiveShell
from generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
import base64
from IPython.core.display import display, HTML

pd.set_option('display.max_columns', None)
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def load_resample_data(id, remove_unknown=True, split_case=20, split_validation=True, policy=1):
    filename = 'data/resample_data_p{POLICY}_{ID}.csv'.format(ID=id,POLICY=policy)
    data_resample = pd.read_csv(filename)
    if remove_unknown:
        data_resample = data_resample[data_resample['label']!=2]
    
    data_resample_train = data_resample[data_resample['event']<split_case]
    data_resample_valid = data_resample[data_resample['event']>=split_case]

    train_aX = data_resample.drop(['label','time','group','timestamp','event'], axis=1).values
    train_ay = data_resample['label'].values.astype(int)
    train_aY = np.eye(2)[train_ay]
    
    train_X = data_resample_train.drop(['label','time','group','timestamp','event'], axis=1).values
    train_y = data_resample_train['label'].values.astype(int)
    valid_X = data_resample_valid.drop(['label','time','group','timestamp','event'], axis=1).values
    valid_y = data_resample_valid['label'].values.astype(int)
    train_Y = np.eye(2)[train_y]
    valid_Y = np.eye(2)[valid_y]

    train_abnormal = data_resample_train[data_resample_train['label']==1]
    train_normal = data_resample_train[data_resample_train['label']==0]
    train_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(train_normal), j=len(train_normal)/len(data_resample_train),k=len(train_abnormal),m=len(train_abnormal)/len(data_resample_train), n=len(data_resample_train))
    
    valid_abnormal = data_resample_valid[data_resample_valid['label']==1]
    valid_normal = data_resample_valid[data_resample_valid['label']==0]
    valid_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(valid_normal), j=len(valid_normal)/len(data_resample_valid),k=len(valid_abnormal),m=len(valid_abnormal)/len(data_resample_valid), n=len(data_resample_valid))
    if split_validation:
        return data_resample, train_X, train_Y, train_y, valid_X, valid_Y, valid_y, train_desc, valid_desc
    else:
        return data_resample, train_aX, train_aY, train_ay

def load_test1_data(id, dropna=True, fillna=0):
    filename = 'data/goodformat_{ID}.csv'.format(ID=id)
    data_raw = pd.read_csv(filename)
    if dropna:
        data_raw = data_raw.dropna()
    else: 
        data_raw = data_raw.fillna(fillna)
    
    data_X = data_raw.drop(['label','time','group','timestamp'], axis=1).values
    data_y = data_raw['label'].values.astype(int)
    data_Y = np.eye(2)[data_y]
    
    data_abnormal = data_raw[data_raw['label']==1]
    data_normal = data_raw[data_raw['label']==0]
    data_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(data_normal), j=len(data_normal)/len(data_raw),k=len(data_abnormal),m=len(data_abnormal)/len(data_raw), n=len(data_raw))
    
    return data_raw, data_X, data_Y, data_y, data_desc

def load_test2_data():
    test_data = pd.read_csv('data/ice1/test/08/08_data.csv')
    test_timeidx = test_data['time']
    tmp = test_data.drop(['time','group'], axis=1)
    test_X = tmp.values
    return test_data, test_X, test_timeidx

    

**Load input data.** 

In [3]:
data_15_resample, _, _, _= load_resample_data(id=15,policy=2,split_validation=False,split_case=8)
data_15_raw, _, _, _= load_resample_data(id=15,policy=0,split_validation=False,split_case=8)

In [4]:
data_15_resample_evt0_3 = data_15_resample[data_15_resample['event']<2]
data_15_resample_pass = data_15_resample_evt0_3[data_15_resample_evt0_3['label']==0]
data_15_resample_fail = data_15_resample_evt0_3[data_15_resample_evt0_3['label']==1]

In [5]:
data_15_raw_evt0_3 = data_15_raw[data_15_raw['event']<1]
data_15_raw_evt0_3_pass = data_15_raw_evt0_3[data_15_raw_evt0_3['label']==0]
data_15_raw_evt0_3_fail = data_15_raw_evt0_3[data_15_raw_evt0_3['label']==1]

In [6]:

# Calculate the feature statistics proto from the datasets and stringify it for use in facets overview
gfsg = GenericFeatureStatisticsGenerator()
proto = gfsg.ProtoFromDataFrames([{'name': 'pass', 'table': data_15_raw_evt0_3_pass},
                                  {'name': 'fail', 'table': data_15_raw_evt0_3_fail}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")

### Global Controls

At the top of the visualization are controls that affect the individual tables.  
The sort-by dropdown changes the sort order for the features in each table. The options are:
    - Feature order: Ordered by their natural order in the feature statistics proto
    - Non-uniformity: Ordered by how non-uniform the distribution of values is (using entropy)
    - Alphabetical
    - Amount missing/zero: Ordered by how many values of the feature are missing or contain the number 0, with the largest amount being first.
    -  Distribution distance (only available when comparing multiple datasets): Ordered by the largest difference between distribution shapes for each feature (using chi-square test for shape).

In [7]:
# Display the facets overview visualization for this data

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
#html = HTML_TEMPLATE.format(protostr=protostr)
#display(HTML(html))

In [8]:
data_15_resample_evt0_3.to_json(orient='records')

'[{"time":"2015-11-04 16:03:00","wind_speed":2.568082834,"generator_speed":1.236889745,"power":2.483754967,"wind_direction":1.079587977,"wind_direction_mean":0.218806513,"yaw_position":-0.614620103,"yaw_speed":-0.213382448,"pitch1_angle":2.28,"pitch2_angle":2.244444444,"pitch3_angle":2.28,"pitch1_speed":-0.2,"pitch2_speed":-0.16,"pitch3_speed":-0.24,"pitch1_moto_tmp":0.01,"pitch2_moto_tmp":-0.139,"pitch3_moto_tmp":-0.139,"acc_x":1.225767053,"acc_y":0.061109469,"environment_tmp":-0.200348758,"int_tmp":-0.422974415,"pitch1_ng5_tmp":1.090769231,"pitch2_ng5_tmp":1.061538462,"pitch3_ng5_tmp":0.952307692,"pitch1_ng5_DC":0.64,"pitch2_ng5_DC":2.84,"pitch3_ng5_DC":0.16,"group":49,"timestamp":1446624180.0,"label":0.0,"event":0},{"time":"2015-11-04 16:03:00","wind_speed":2.405812317,"generator_speed":1.263480184,"power":2.495776779,"wind_direction":1.034702701,"wind_direction_mean":1.482811064,"yaw_position":-0.614620103,"yaw_speed":-0.17849873,"pitch1_angle":2.057777778,"pitch2_angle":2.03111111

In [10]:


HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = JSON.parse('{jsonstr}');
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=data_15_resample_evt0_3)
display(HTML(html))